<a href="https://colab.research.google.com/github/ChaitanyaGadde98/national-anthems-classification-nlp/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install "tensorflow>=1.15,<2.0"
!pip install --upgrade tensorflow-hub
!pip install pandas
!pip install googledrivedownloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow<2.0,>=1.15 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0rc0, 2.12.0rc1)
ERROR: No matching distribution found for tensorflow<2.0,>=1.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

# import necessary libraries
import tensorflow_hub as hub
  
# Load pre-trained universal sentence encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  
# Sentences for which you want to create embeddings,
# passed as an array in embed()
Sentences = [
    "How old are you",
    "What is your age",
    "I love to watch Television",
    "I am wearing a wrist watch"
]
embeddings = embed(Sentences)
  
# Printing embeddings of each sentence
print(embeddings)
  
# To print each embeddings along with its corresponding 
# sentence below code can be used.
for i in range(len(Sentences)):
    print(Sentences[i])
    print(embeddings[i])

tf.Tensor(
[[-0.06045125 -0.00204541  0.02656925 ...  0.00764413 -0.02669661
   0.05110302]
 [-0.08415681 -0.08687922  0.03446117 ... -0.01439389 -0.04546221
   0.03639964]
 [ 0.0816019  -0.01570276 -0.05659244 ... -0.07133698  0.11040761
  -0.0071095 ]
 [-0.00369539  0.03064634 -0.05556112 ...  0.01751423  0.0316496
  -0.05139377]], shape=(4, 512), dtype=float32)
How old are you
tf.Tensor(
[-0.06045125 -0.00204541  0.02656925  0.02483987 -0.09206301  0.01760055
 -0.00284112 -0.00158421 -0.00648677  0.05091851 -0.01242187  0.0196162
  0.01128109 -0.04664037 -0.00683796 -0.04489101 -0.08644009  0.00650328
 -0.05588394 -0.09097648 -0.01062818  0.00379061  0.00729432 -0.03010405
 -0.02222248  0.01104948  0.02357136 -0.01104616  0.0022157   0.01935781
  0.02867085 -0.07079582  0.05549346  0.00179186 -0.02146762 -0.00257316
 -0.00539005 -0.03071919  0.00420406  0.0023769  -0.03511341  0.01090724
  0.01152228  0.10254128  0.04378743 -0.06699706 -0.10248536  0.00487916
  0.06548391 -0.0132634

In [7]:
!ls

 sample_data  'view?usp=share_link'


In [12]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import gdown

url = 'https://drive.google.com/file/d/1rUjeCOcuWUAf-SoCfXQPehBu8Uneyt9p'
output = 'anthems.txt'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.9/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1rUjeCOcuWUAf-SoCfXQPehBu8Uneyt9p
To: /content/anthems.txt
74.2kB [00:00, 33.8MB/s]


'anthems.txt'

In [11]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1rUjeCOcuWUAf-SoCfXQPehBu8Uneyt9p',
                                    dest_path='anthems.csv')

FileNotFoundError: ignored

In [14]:
!ls

 anthems.txt   sample_data  'view?usp=share_link'


In [15]:
!cat anthems.txt

<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style nonce="CYDNIV5Ed7mQzGIdaTivGw">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>anthems.cs